In [ ]:
!pip install -q streamlit requests pandas trafilatura cloudscraper seaborn pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 837.9/837.9 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 9.4 MB/s eta 0:00:00


In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import requests
import time
import trafilatura
import cloudscraper
import re
import matplotlib.pyplot as plt
import seaborn as sns
import math
from datetime import datetime, timedelta

# --- 1. Load Colab Secrets ---
def get_colab_secrets():
    try:
        from google.colab import userdata
        id_val = userdata.get('ADZUNA_APP_ID')
        key_val = userdata.get('ADZUNA_APP_KEY')
        return id_val, key_val
    except:
        return "", ""

secret_id, secret_key = get_colab_secrets()

# --- 2. UI Setup ---
st.set_page_config(page_title="Job Skill Analyzer", layout="wide")
st.title("🎯 Job Skill Analyzer: Data Science & Analytics")

# --- 3. Sidebar ---
with st.sidebar:
    st.header("🔑 API Credentials")
    app_id = st.text_input("Adzuna APP ID", value=secret_id if secret_id else "")
    app_key = st.text_input("Adzuna APP KEY", type="password", value=secret_key if secret_key else "")

    st.header("🔍 Filters")
    country = st.selectbox("Country", ["us", "uk", "ca", "au"], index=0)
    city_options = ["Baltimore", "Washington DC", "Ellicott City", "Columbia", "Towson", "Silver Spring"]
    selected_cities = st.multiselect("Select Cities", options=city_options, default=["Baltimore"])

    keywords_input = st.text_input("Keywords", "data scientist, data analyst")
    keywords = [k.strip() for k in keywords_input.split(",")]

    st.header("⚙️ Settings")
    days_back = st.slider("Days Back", 7, 90, 60)
    max_pages = st.number_input("Max Pages", 1, 10, 5)

# --- 4. Taxonomy ---
SKILL_KEYWORDS = {
    'Programming': ['Python', 'SQL', ' R ', 'SAS', 'Stata', 'Julia', 'C++', 'Java', 'Scala', 'Go', 'Bash', 'Shell'],
    'Cloud & Big Data': ['AWS', 'Azure', 'GCP', 'Google Cloud', 'Snowflake', 'Databricks', 'Spark', 'Hadoop', 'Kafka', 'Redshift', 'BigQuery', 'Athena', 'Glue', 'Terraform', 'Airflow'],
    'Databases': ['PostgreSQL', 'MySQL', 'MongoDB', 'NoSQL', 'SQL Server', 'Oracle', 'Cassandra'],
    'ML & AI': ['Machine Learning', 'Deep Learning', 'Reinforcement Learning', 'NLP', 'Natural Language Processing', 'Computer Vision', 'Generative AI', 'GenAI', 'LLM', 'GPT', 'Scikit-learn', 'TensorFlow', 'PyTorch', 'Keras', 'XGBoost', 'LightGBM', 'CatBoost','AI'],
    'Stats & Research': ['Statistics', 'statisti', 'Biostatistics', 'Causal Inference', 'Epidemiology', 'Econometrics', 'Bayesian', 'Survival Analysis', 'Longitudinal', 'Time Series', 'A/B Testing', 'Experimental Design', 'Propensity Score', 'Clinical Trials', 'Regression', 'Hypothesis Testing', 'RCT', 'GIS', 'Geographic Information Systems', 'Spatial'],
    'Visualization & BI': ['Tableau', 'Power BI', 'Looker', 'Qlik', 'Matplotlib', 'Seaborn', 'Plotly', 'Shiny', 'D3.js', 'Insights'],
    'Engineering & DevOps': ['Git', 'GitHub', 'CI/CD', 'Docker', 'Kubernetes', 'MLOps', 'Agile', 'Scrum', 'DevOps'],
    'Degree': ['PhD', 'Ph.D.', 'Doctorate', 'Master', 'M.S.', 'MSc', 'MPH', 'MBA', 'Bachelor']

} #soft skill (leadship, ), how many years, which industries, fuzzy matching, ETL, data analysis, dashboard, define kpi(business matrics)

def is_relevant_title(title):
    target = ["data scientist", "senior data scientist", "sr. data scientist", "lead data scientist", "principal data scientist", "staff data scientist", "data analyst", "senior data analyst"]
    exclude = ["architect", "engineer", "manager", "consultant", "director", "vp", "vice president", "head of", "marketing", "sales", "product"]
    t = title.lower()
    return any(k in t for k in target) and not any(k in t for k in exclude)

def find_skills_refined(text):
    if pd.isna(text): return []
    res = []
    for cat, kws in SKILL_KEYWORDS.items():
        for k in kws:
            pattern = r'\b' + re.escape(k.strip()) + r'\b'
            if re.search(pattern, text, re.I):
                val = k.strip().replace('Ph.D.', 'PhD').replace('M.S.', 'Master')
                res.append(val)
    return list(set(res))

def fetch_full_description(url):
    scraper = cloudscraper.create_scraper(browser={'browser': 'chrome', 'platform': 'windows', 'desktop': True})
    try:
        response = scraper.get(url, timeout=15)
        if response.status_code == 200:
            content = trafilatura.extract(response.text)
            return content.strip() if content and len(content) > 300 else None
    except: pass
    return None

# --- 5. Main Execution ---
if st.sidebar.button("🚀 Start Deep Analysis"):
    if not app_id or not app_key:
        st.error("Missing Credentials.")
    else:
        st.warning("⏳ **Deep Scraping Active:** Fetching full descriptions for accuracy. This will take a few minutes...")

        all_jobs = []
        cutoff_date = datetime.now() - timedelta(days=days_back)

        with st.status("Fetching Data...", expanded=True) as status:
            for city in selected_cities:
                for kw in keywords:
                    for page in range(1, max_pages + 1):
                        params = {"app_id": app_id, "app_key": app_key, "what": kw, "where": city, "results_per_page": 50, "sort_by": "date"}
                        try:
                            r = requests.get(f"https://api.adzuna.com/v1/api/jobs/{country}/search/{page}", params=params)
                            data = r.json().get("results", [])
                            if not data: break
                            for job in data:
                                if not is_relevant_title(job.get("title")): continue
                                full_desc = fetch_full_description(job.get("redirect_url"))
                                all_jobs.append({
                                    "Title": job.get("title"),
                                    "Company": job.get("company", {}).get("display_name"),
                                    "Description": full_desc if full_desc else job.get("description", ""),
                                    "URL": job.get("redirect_url")
                                })
                                time.sleep(0.5)
                        except: break
            status.update(label="Scanning Complete!", state="complete", expanded=False)

        if all_jobs:
            df = pd.DataFrame(all_jobs)
            df['skills'] = df['Description'].apply(find_skills_refined)
            df_effective = df[df['skills'].map(len) > 0].copy()
            n_sample = len(df_effective)

            # --- 全新加強版視覺化 ---
            st.subheader("📊 Skill Analysis Results")

            all_found = []
            for s_list in df_effective['skills']:
                for s in s_list:
                    for cat, ks in SKILL_KEYWORDS.items():
                        if s in [k.strip().replace('Ph.D.', 'PhD').replace('M.S.', 'Master') for k in ks]:
                            all_found.append({'Category': cat, 'Skill': s})

            stats_df = pd.DataFrame(all_found)
            cols = 2
            rows = math.ceil(len(SKILL_KEYWORDS) / cols)

            # --- 關鍵修正：強制放大所有字體 ---
            # 直接修改全局參數
            plt.rcParams.update({
                'font.size': 22,
                'axes.titlesize': 28,
                'axes.labelsize': 20,
                'ytick.labelsize': 20,
                'xtick.labelsize': 18,
                'font.weight': 'bold'
            })

            fig, axes = plt.subplots(rows, cols, figsize=(22, rows * 7))
            axes = axes.flatten()
            palette = sns.color_palette("viridis", len(SKILL_KEYWORDS))

            for i, cat in enumerate(SKILL_KEYWORDS.keys()):
                ax = axes[i]
                cat_data = stats_df[stats_df['Category'] == cat]
                if not cat_data.empty:
                    counts = cat_data['Skill'].value_counts().reset_index()
                    counts.columns = ['Skill', 'Count']
                    sns.barplot(data=counts, x='Count', y='Skill', ax=ax, color=palette[i])
                    ax.set_title(f'{cat}', pad=25) # 類別標題
                    ax.set_xlabel('Mentions', labelpad=15)
                    ax.set_ylabel('')
                    ax.set_xlim(0, n_sample * 1.25)

                    # 數據標籤加強
                    for p in ax.patches:
                        w = int(p.get_width())
                        ax.annotate(f'{w} ({(w/n_sample)*100:.1f}%)',
                                    (w, p.get_y() + p.get_height()/2),
                                    xytext=(15, 0), textcoords='offset points',
                                    fontweight='bold', fontsize=18, va='center')
                else:
                    ax.set_title(f'{cat} (No Data)')

            for j in range(i+1, len(axes)): fig.delaxes(axes[j])

            # 總標題加強
            plt.suptitle(f'Market Analysis: Data Science & Analytics\nLocations: {", ".join(selected_cities)}\n(N = {n_sample} Jobs with Skills | Total = {len(df)})',
                         fontsize=34, fontweight='bold', y=0.98, color='#1a2a6c')

            plt.tight_layout(rect=[0, 0.03, 1, 0.94])
            st.pyplot(fig)

            st.download_button("📥 Download Results (CSV)", data=df.to_csv(index=False).encode('utf-8-sig'), file_name="job_analysis.csv")

Writing app.py


In [ ]:
# 2. 設定 Token (把引號內換成你的)
from pyngrok import ngrok
ngrok.set_auth_token("API check")
# Start Streamlit server on a specific port
!nohup streamlit run app.py --server.port 5011 &

# Start ngrok tunnel to expose the Streamlit server
ngrok_tunnel = ngrok.connect(addr='5011', proto='http', bind_tls=True)

# Print the URL of the ngrok tunnel
print(' * Tunnel URL:', ngrok_tunnel.public_url)

nohup: appending output to 'nohup.out'
 * Tunnel URL: https://dc4a-34-66-251-13.ngrok-free.app


In [ ]:
# 1. 斷開所有作用中的隧道
try:
    tunnels = ngrok.get_tunnels()
    for tunnel in tunnels:
        print(f"Disconnecting: {tunnel.public_url}")
        ngrok.disconnect(tunnel.public_url)
except:
    pass

# 2. 殺掉所有 ngrok 與 streamlit 的背景進程
!pkill ngrok
!pkill streamlit

Disconnecting: https://6ae0397e0fa8.ngrok-free.app
Disconnecting: https://58d2b7dbfd0a.ngrok-free.app
